# Ceneo Scraper

# Biblioteki

In [1]:
import requests
from bs4 import BeautifulSoup


## Wysyłanie do serwera żądania dostępu do zasobu

In [2]:
product_id = "94103927"
url = f"https://www.ceneo.pl/{product_id}/opinie-2"
response = requests.get(url)

In [ ]:
page_dom = BeautifulSoup(response.text, 'html.parser')
print(type(page_dom))
opinions = page_dom.select("div.js_product-review")
print(type(opinions))
opinion = page_dom.select_one("div.js_product-review")
print(type(opinion))
print(opinion)


## Ekstrakcja składowych pojedynczej opinii


|Składowa           |Zmienna                |Selektor
|id opinii          |opinion_id             |["data-entry-id"]
|autor              |author_name            |span.user-post__author-name
|rekomendacja       |recommendation         |span.user-post__author-recomendation > em
|gwiazdki           |rating                 |span.user-post__score-count
|treść              |content                |div.user-post__text
|lista zalet        |pros                   |~div.review-feature__title review-feature__title--positives
|lista wad          |cons                   |~div.review-feature__title review-feature__title--negatives
|vote up            |useful                 |button.vote-yes > span|useful|
|vote down          |useless                |button.vote-no > span|useless|
|data wystawienia   |publish_date           |span.user-post__published > time:nth-child(1)['datetime']
|data zakupu        |purchase_date          |span.user-post__published > time:nth-child(2)['datetime']



In [ ]:
opinion_id = opinion["data-entry-id"]
author_name = opinion.select_one("span.user-post__author-name")
recommendation_parent = opinion.select_one("div.user-post__author-recomendation")
recommendation = recommendation_parent.select("b:nth-child.user-post__author-recomendation")
rating = opinion.select_one("div.user-post__score")
content = opinion.select_one(".user-post__text")
pros = opinion.select_one(".review-feature__title review-feature__title--positives")